Reproduce results of the 3D semantic instance segmentation with Mask3D

Refer to:

paper: 
[Mask3D for 3D Semantic Instance Segmentation ](https://arxiv.org/abs/2210.03105)

Github: 
Mask3D: https://github.com/JonasSchult/Mask3D



# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/Mask3D
%ls

/content/drive/MyDrive/stanford/CS 231A/project/Mask3D
 3DSceneGraph_medium.zip   final.ipynb
 3DSceneGraph_tiny.zip     LICENSE
'3D scene.ipynb'           main_instance_segmentation.py
 area1_from_scratch.ckpt   models/
 area2_from_scratch.ckpt   note.txt
 area3_from_scratch.ckpt   Pointnet2_PyTorch/
 area4_from_scratch.ckpt   README.md
 area6_from_scratch.ckpt   requirements1.txt
 benchmark/                requirements.txt
 cfg.pickle                run.ipynb
 conf/                     saved/
'Copy of final1.ipynb'     scannet/
'Copy of final.ipynb'      scannet_val.ckpt
'Copy of run.ipynb'        scripts/
 data/                     setup.sh
 dataset/                  split.py
 datasets/                 stpls3d_val1.ckpt
 docs/                     Synthetic_v3_InstanceSegmentation/
 download_scannet.py       Synthetic_v3_InstanceSegmentation.zip
 download_scannetv2.py     third_party/
 download_some_scan.sh     trainer/
 eval_output/              utils/


In [ ]:
!bash setup.sh

setup.sh: line 2: $'\r': command not found
setup.sh: line 3: conda: command not found
setup.sh: line 4: conda: command not found
setup.sh: line 5: $'\r': command not found
setup.sh: line 6: conda: command not found
setup.sh: line 7: conda: command not found
setup.sh: line 8: $'\r': command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu116.html
setup.sh: line 11: $'\r': command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
setup.sh: line 14: $'\r': command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking i

In [ ]:
!pip install --no-dependencies omegaconf==2.0.6 hydra-core==1.0.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --no-dependencies 'git+https://github.com/facebookresearch/detectron2.git@710e7795d0eeadf9def0e7ef957eea13532e34cf'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git (to revision 710e7795d0eeadf9def0e7ef957eea13532e34cf) to /tmp/pip-req-build-41cpl47y
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-41cpl47y
  Running command git rev-parse -q --verify 'sha^710e7795d0eeadf9def0e7ef957eea13532e34cf'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 710e7795d0eeadf9def0e7ef957eea13532e34cf
  Running command git checkout -q 710e7795d0eeadf9def0e7ef957eea13532e34cf
  Resolved https://github.com/facebookresearch/detectron2.git to commit 710e7795d0eeadf9def0e7ef957eea13532e34cf
  Preparing metadata (setup.py) ... done


install `MinkowskiEngine` manually

In [ ]:
!sudo apt-get install libopenblas-dev
!pip install git+https://github.com/NVIDIA/MinkowskiEngine.git -v 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.3.8+ds-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Using pip 22.0.4 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/MinkowskiEngine.git to /tmp/pip-req-build-5_9kkjf2
  Running command git version
  git version 2.25.1
  Running command git clone --filter=blob:none https://github.com/NVIDIA/MinkowskiEngine.git /tmp/pip-req-build-5_9kkjf2
  Cloning into '/tmp/pip-req-build-5_9kkjf2'...
  Updating files:  92% (226/244)
  Updating files:  93% (227/244)
  Updating files:  94% (230/244)
  Updating files:  95% (232/244)
  Updating files:  96% (235/244)
  Updating files:  97% (237/244)
  Updating files:  98% (240/244)
  Updating files:  99% (242/244)
  Updating files:

In [ ]:
!pip install fvcore==0.1.5.post20220512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytorch-lightning==1.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Scannet

## Prepare Dataset

Create Surface mesh segmentation files (`*.segs.json`) for testing

In [ ]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/ScanNet/Segmentator

In [ ]:
import subprocess

In [ ]:
subprocess.call(['./segmentator', "/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet/scans_test/scene0707_00/scene0707_00_vh_clean_2.ply"])

In [ ]:
import os
from tqdm import tqdm

test_list = os.listdir("/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet/scans_test")

path = "/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet/scans_test/"
for idx, test_i in tqdm(enumerate(test_list)):
  filename = path + test_i + '/' + test_i + '_vh_clean_2.ply'
  res = subprocess.call(['./segmentator', filename]) 
  print(idx, test_i, res)

  

In [ ]:
# transfer files to another folder ./data/raw/scannet_test_segments
import shutil
path = "/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet/scans_test/"
dst_path = "/content/drive/MyDrive/stanford/CS 231A/data/raw/scannet_test_segments/"
test_list = os.listdir(path)
for idx, test_i in tqdm(enumerate(test_list)):
  filename = test_i + '_vh_clean_2.0.010000.segs.json'
  shutil.copyfile(src=path + test_i + '/' + filename, dst=dst_path + filename)
  print(idx, test_i)

In [ ]:
! pip install open3d

In [ ]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/Mask3D

In [ ]:
!python datasets/preprocessing/scannet_preprocessing.py preprocess \
--data_dir="/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet" \
--save_dir="../../data/processed/scannet" \
--git_repo="/content/drive/MyDrive/stanford/CS 231A/project/ScanNet" \
# --scannet200=false

## Evaluate

In [ ]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

validation 1 - 156

In [ ]:
!python main_instance_segmentation.py \
general.experiment_name="validation_query_150_topk_500_dbscan_0.95" \
general.project_name="scannet_eval" \
general.checkpoint='/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet_val.ckpt' \
general.train_mode=False \
general.eval_on_segments="true" \
general.train_on_segments="true" \
model.num_queries=150 \
general.topk_per_image=500 \
general.use_dbscan="true" \
general.dbscan_eps=0.95

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name validation_query_150_topk_500_dbscan_0.95
project_name scannet_eval
train_mode False
checkpoint /content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet_val.ckpt
eval_on_segments True
train_on_segments True
num_queries 150
topk_per_image 500
use_dbscan True
dbscan_eps 0.95
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 3372033128
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3372033128
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my r

validation 157-312

In [ ]:
!python main_instance_segmentation.py \
general.experiment_name="validation_query_150_topk_500_dbscan_0.95" \
general.project_name="scannet_eval" \
general.checkpoint='/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet_val.ckpt' \
general.train_mode=False \
general.eval_on_segments="true" \
general.train_on_segments="true" \
model.num_queries=150 \
general.topk_per_image=500 \
general.use_dbscan="true" \
general.dbscan_eps=0.95

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name validation_query_150_topk_500_dbscan_0.95
project_name scannet_eval
train_mode False
checkpoint /content/drive/MyDrive/stanford/CS 231A/project/Mask3D/scannet_val.ckpt
eval_on_segments True
train_on_segments True
num_queries 150
topk_per_image 500
use_dbscan True
dbscan_eps 0.95
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 3118187412
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3118187412
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my r

# STPLS3D

## Prepare dataset

Refer to [STPLS3D: A Large-Scale Synthetic and Real Aerial Photogrammetry 3D Point Cloud Dataset](https://github.com/meidachen/STPLS3D#42-instance-segmentation)

In [ ]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/STPLS3D/HAIS/data

/content/drive/MyDrive/stanford/CS 231A/project/STPLS3D/HAIS/data


In [ ]:
!python prepare_data_inst_instance_stpls3d.py

Streaming output truncated to the last 5000 lines.
unique insance: 0
unique semantic: 0

range z is smaller than threshold 
7_points_GTv3_437_inst_nostuff
unique insance: 0
unique semantic: 1

range z is smaller than threshold 
7_points_GTv3_438_inst_nostuff
unique insance: 0
unique semantic: 1

range z is smaller than threshold 
7_points_GTv3_440_inst_nostuff
unique insance: 3
unique semantic: 2

range z is smaller than threshold 
7_points_GTv3_442_inst_nostuff
range z is smaller than threshold 
7_points_GTv3_444_inst_nostuff
unique insance: 9
unique semantic: 3

range z is smaller than threshold 
7_points_GTv3_445_inst_nostuff
unique insance: 4
unique semantic: 2

range z is smaller than threshold 
7_points_GTv3_446_inst_nostuff
unique insance: 0
unique semantic: 1

range z is smaller than threshold 
7_points_GTv3_449_inst_nostuff
unique insance: 5
unique semantic: 2

range z is smaller than threshold 
7_points_GTv3_450_inst_nostuff
unique insance: 6
unique semantic: 2

range z is sm

In [ ]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/Mask3D

'/content/drive/MyDrive/stanford/CS 231A/project/Mask3D'

In [ ]:
!python datasets/preprocessing/stpls3d_preprocessing.py preprocess \
--data_dir="/content/drive/MyDrive/stanford/CS 231A/project/STPLS3D/HAIS/dataset/Synthetic_v3_InstanceSegmentation" \
--save_dir="/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/data/processed/stpls3d"

2023-03-23 15:15:35.772 | INFO     | __main__:preprocess:58 - Tasks for train: 4762
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    2.8s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    3.3s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    3.6s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    4.1s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    5.2s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    5.7s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    6.5s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:   15.8s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:  6.0min
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:  6.4min
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:  6.4min
[Parallel(n_jobs=12)]: Done 176 t

## Evaluation

Evaluate on validation

In [ ]:
!export OMP_NUM_THREADS=3

CURR_DBSCAN=14.0
CURR_TOPK=750
CURR_QUERY=160
CURR_SIZE=54

!python main_instance_segmentation.py \
general.experiment_name="validation_query_160_topk_750_dbscan_14_size_54" \
general.project_name="stpls3d_eval" \
data/datasets=stpls3d \
general.num_targets=15 \
data.num_labels=15 \
data.voxel_size=0.333 \
data.num_workers=10 \
data.cache_data=true \
data.cropping_v1=false \
general.reps_per_epoch=100 \
model.num_queries=160 \
general.on_crops=true \
model.config.backbone._target_=models.Res16UNet18B \
general.train_mode=false \
general.checkpoint="stpls3d_val1.ckpt" \
data.crop_length=54 \
general.eval_inner_core=50.0 \
general.topk_per_image=750 \
general.use_dbscan=true \
general.dbscan_eps=14

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name validation_query_160_topk_750_dbscan_14_size_54
project_name stpls3d_eval
train_mode False
checkpoint stpls3d_val1.ckpt
eval_on_segments False
train_on_segments False
num_queries 160
topk_per_image 750
use_dbscan True
dbscan_eps 14
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 3240705251
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3240705251
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You ch

# S3DIS

In [ ]:
%cd /content/drive/MyDrive/stanford/CS 231A/project/Mask3D

/content/drive/MyDrive/stanford/CS 231A/project/Mask3D


In [ ]:
%ls

 3DSceneGraph_medium.zip         models/
 3DSceneGraph_tiny.zip           note.txt
'3D scene.ipynb'                 Pointnet2_PyTorch/
 area3_from_scratch.ckpt         README.md
 benchmark/                      requirements1.txt
 cfg.pickle                      requirements.txt
 conf/                           run.ipynb
'Copy of final.ipynb'            saved/
'Copy of run.ipynb'              scannet/
 data/                           scannet_val.ckpt
 dataset/                        scripts/
 datasets/                       setup.sh
 docs/                           split.py
 download_scannet.py             stpls3d_val1.ckpt
 download_scannetv2.py           Synthetic_v3_InstanceSegmentation/
 download_some_scan.sh           Synthetic_v3_InstanceSegmentation.zip
 eval_output/                    third_party/
 final.ipynb                     trainer/
 LICENSE                         utils/
 main_instance_segmentation.py


In [ ]:
!python datasets/preprocessing/s3dis_preprocessing.py preprocess \
--data_dir="/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2" \
--save_dir="/content/drive/MyDrive/stanford/CS 231A/project/Mask3D/data/processed/s3dis" \


Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/auditorium_2/Annotations/chair_171.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/office_8/Annotations/wall_1.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/office_11/Annotations/clutter_2.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/auditorium_2/Annotations/chair_175.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/auditorium_1/Annotations/chair_92.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/office_10/Annotations/wall_2.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/auditorium_1/Annotations/chair_93.txt
/content/drive/MyDrive/stanford/CS 231A/project/Stanford3dDataset_v1.2/Area_2/auditorium_2/Annotations/chair_179.txt
/content/drive/MyDrive/stanford

### Evaluation

In [ ]:
%pwd

'/content/drive/MyDrive/stanford/CS 231A/project/Mask3D'

#### Area 1

In [ ]:
%ls

 3DSceneGraph_medium.zip   final.ipynb
 3DSceneGraph_tiny.zip     LICENSE
'3D scene.ipynb'           main_instance_segmentation.py
 area1_from_scratch.ckpt   models/
 area2_from_scratch.ckpt   note.txt
 area3_from_scratch.ckpt   Pointnet2_PyTorch/
 area4_from_scratch.ckpt   README.md
 area6_from_scratch.ckpt   requirements1.txt
 benchmark/                requirements.txt
 cfg.pickle                run.ipynb
 conf/                     saved/
'Copy of final1.ipynb'     scannet/
'Copy of final.ipynb'      scannet_val.ckpt
'Copy of run.ipynb'        scripts/
 data/                     setup.sh
 dataset/                  split.py
 datasets/                 stpls3d_val1.ckpt
 docs/                     Synthetic_v3_InstanceSegmentation/
 download_scannet.py       Synthetic_v3_InstanceSegmentation.zip
 download_scannetv2.py     third_party/
 download_some_scan.sh     trainer/
 eval_output/              utils/


In [ ]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

CURR_AREA=1  # set the area number accordingly [1,6]
CURR_DBSCAN=0.6
CURR_TOPK=-1
CURR_QUERY=100

!python main_instance_segmentation.py \
general.project_name="s3dis_eval" \
general.experiment_name="area{CURR_AREA}_from_scratch_eps_{CURR_DBSCAN}_topk_{CURR_TOPK}_q_{CURR_QUERY}" \
general.checkpoint="area{CURR_AREA}_from_scratch.ckpt" \
general.train_mode=false \
data.batch_size=4 \
data/datasets=s3dis \
general.num_targets=14 \
data.num_labels=13 \
general.area={CURR_AREA} \
model.num_queries={CURR_QUERY} \
general.topk_per_image={CURR_TOPK} \
general.use_dbscan=true \
general.dbscan_eps={CURR_DBSCAN}

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name area1_from_scratch_eps_0.6_topk_-1_q_100
project_name s3dis_eval
train_mode False
checkpoint area1_from_scratch.ckpt
eval_on_segments False
train_on_segments False
num_queries 100
topk_per_image -1
use_dbscan True
dbscan_eps 0.6
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 1847778852
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1847778852
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose

#### Area 2

In [5]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

CURR_AREA=2  # set the area number accordingly [1,6]
CURR_DBSCAN=0.6
CURR_TOPK=-1
CURR_QUERY=80

!python main_instance_segmentation.py \
general.project_name="s3dis_eval" \
general.experiment_name="area{CURR_AREA}_from_scratch_eps_{CURR_DBSCAN}_topk_{CURR_TOPK}_q_{CURR_QUERY}" \
general.checkpoint="area{CURR_AREA}_from_scratch.ckpt" \
general.train_mode=false \
data.batch_size=2 \
data/datasets=s3dis \
general.num_targets=14 \
data.num_labels=13 \
general.area={CURR_AREA} \
model.num_queries={CURR_QUERY} \
general.topk_per_image={CURR_TOPK} \
general.use_dbscan=true \
general.dbscan_eps={CURR_DBSCAN}

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name area2_from_scratch_eps_0.6_topk_-1_q_80
project_name s3dis_eval
train_mode False
checkpoint area2_from_scratch.ckpt
eval_on_segments False
train_on_segments False
num_queries 80
topk_per_image -1
use_dbscan True
dbscan_eps 0.6
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 2568887749
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 2568887749
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: Traceback (most recent call last):
  File "/u

#### Area 3

In [ ]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

CURR_AREA=1  # set the area number accordingly [1,6]
CURR_DBSCAN=0.6
CURR_TOPK=-1
CURR_QUERY=100

!python main_instance_segmentation.py \
general.project_name="s3dis_eval" \
general.experiment_name="area3_from_scratch_eps_0.6_topk_-1_q_100" \
general.checkpoint="area3_from_scratch.ckpt" \
general.train_mode=false \
data.batch_size=4 \
data/datasets=s3dis \
general.num_targets=14 \
data.num_labels=13 \
general.area=3 \
model.num_queries=100 \
general.topk_per_image=-1 \
general.use_dbscan=true \
general.dbscan_eps=0.6

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name area3_from_scratch_eps_0.6_topk_-1_q_100
project_name s3dis_eval
train_mode False
checkpoint area3_from_scratch.ckpt
eval_on_segments False
train_on_segments False
num_queries 100
topk_per_image -1
use_dbscan True
dbscan_eps 0.6
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 116465074
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 116465074
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "

#### Area 4

In [ ]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

CURR_AREA=4  # set the area number accordingly [1,6]
CURR_DBSCAN=0.6
CURR_TOPK=-1
CURR_QUERY=100

!python main_instance_segmentation.py \
general.project_name="s3dis_eval" \
general.experiment_name="area{CURR_AREA}_from_scratch_eps_{CURR_DBSCAN}_topk_{CURR_TOPK}_q_{CURR_QUERY}" \
general.checkpoint="area{CURR_AREA}_from_scratch.ckpt" \
general.train_mode=false \
data.batch_size=4 \
data/datasets=s3dis \
general.num_targets=14 \
data.num_labels=13 \
general.area={CURR_AREA} \
model.num_queries={CURR_QUERY} \
general.topk_per_image={CURR_TOPK} \
general.use_dbscan=true \
general.dbscan_eps={CURR_DBSCAN}

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name area4_from_scratch_eps_0.6_topk_-1_q_100
project_name s3dis_eval
train_mode False
checkpoint area4_from_scratch.ckpt
eval_on_segments False
train_on_segments False
num_queries 100
topk_per_image -1
use_dbscan True
dbscan_eps 0.6
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 3701589427
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3701589427
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose

#### Area 6

In [ ]:
!export OMP_NUM_THREADS=3  # speeds up MinkowskiEngine

CURR_AREA=6  # set the area number accordingly [1,6]
CURR_DBSCAN=0.6
CURR_TOPK=-1
CURR_QUERY=100

!python main_instance_segmentation.py \
general.project_name="s3dis_eval" \
general.experiment_name="area{CURR_AREA}_from_scratch_eps_{CURR_DBSCAN}_topk_{CURR_TOPK}_q_{CURR_QUERY}" \
general.checkpoint="area{CURR_AREA}_from_scratch.ckpt" \
general.train_mode=false \
data.batch_size=4 \
data/datasets=s3dis \
general.num_targets=14 \
data.num_labels=13 \
general.area={CURR_AREA} \
model.num_queries={CURR_QUERY} \
general.topk_per_image={CURR_TOPK} \
general.use_dbscan=true \
general.dbscan_eps={CURR_DBSCAN}

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
experiment_name area6_from_scratch_eps_0.6_topk_-1_q_100
project_name s3dis_eval
train_mode False
checkpoint area6_from_scratch.ckpt
eval_on_segments False
train_on_segments False
num_queries 100
topk_per_image -1
use_dbscan True
dbscan_eps 0.6
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 3455049406
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3455049406
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my resu